# ABS Quarterly Job Vacancies 6354

## Python set-up

In [1]:
# system imports
import textwrap
from typing import Any, cast

# analytic imports
import pandas as pd
from readabs import metacol
import readabs as ra

# local imports
from abs_helper import get_abs_data
from abs_plotting import plot_rows_individually
from plotting import line_plot, plot_covid_recovery
from abs_plotting import plot_rows_individually
from plotting import line_plot, plot_covid_recovery, finalise_plot

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, RECENT = get_abs_data("6354.0")
plot_times = None, RECENT
plot_tags = "", "-recent"

In [3]:
# list tables
textwrap.wrap(", ".join(abs_dict.keys()), width=80)

['6354001, 6354002, 6354003, 6354004']

## Plot

### Headline charts

In [4]:
def plot_headline() -> None:
    """Produce headline charts."""

    # get mean quarterly Labour force numbers for Australia
    lfs, _ = ra.read_abs_series("6202.0", "A84423047L", single_excel_only="6202001")
    lf = ra.monthly_to_qtly(lfs["A84423047L"], q_ending="NOV", f="mean")

    # get headline job vacancy rate and job vacancies
    stype = "Seasonally Adjusted"
    selector: dict[str, str] = {
        stype: metacol.stype,
        "6354001": metacol.table,
        "Job Vacancies ;  Australia ;": metacol.did,
    }
    table, ident, units = ra.find_abs_id(meta, selector)
    jva = abs_dict[table][ident]
    # surface missing data in the job vacancy series
    jva = jva.reindex(pd.period_range(start=jva.index[0], end=lf.index[-1]))
    jvr = jva / lf * 100

    # plotting
    common: dict[str, Any] = {
        "rfooter": source,
        "lfooter": f"Australia. {stype.capitalize()} series. ",
        "show": SHOW,
    }
    for data, ylabel, title in zip(
        (jvr, jva),
        ("Job Vacancy Rate (%)", "Thousands"),
        ("Job Vacancy Rate", "Job Vacancies"),
    ):
        also = common.copy()
        if "Rate" in title:
            also["lfooter"] += "Job Vacancy Rate = Job Vacancies / Labour Force. "
        also["ylabel"] = ylabel
        also["title"] = title

        line_plot(
            data,
            dropna=False,
            **also,
        )
        plot_covid_recovery(
            data,
            start_r="2009Q4",
            end_r="2019Q4",
            tags="-covid",
            **also,
        )


plot_headline()

### Industry sector charts

In [5]:
def jv_rates_by_sector():

    # number of employed persons by industry sector
    table = "6291004"
    detail_lf_cat = "6291.0.55.001"
    d, m = ra.read_abs_cat(detail_lf_cat, single_excel_only=table)
    rows = m[m[metacol.stype] == "Seasonally Adjusted"]
    denom_units = rows[metacol.unit].unique()
    denominator = d[table][rows[metacol.id]]
    denominator.columns = rows[metacol.did].str.split(";").str[0].str.strip()

    # number of job vacancies by industry sector
    table = "6354004"
    rows = meta[
        (meta[metacol.table] == table)
        & (~meta[metacol.did].str.contains("Standard Error"))
    ]
    num_units = rows[metacol.unit].unique()
    numerator = abs_dict[table][rows[metacol.id]]
    numerator.columns = rows[metacol.did].str.split(";").str[1].str.strip()

    # calculate vacancy rates (of a sort)
    rate = {}
    change = {}
    for col in numerator.columns:
        if col not in denominator.columns:
            continue
        rate[col] = (numerator[col] / denominator[col] * 100).dropna()
        base = rate[col]["2009Q4":"2019Q4"].mean()
        last2 = rate[col].iloc[-2]
        last = rate[col].iloc[-1]
        change[col] = pd.Series(
            [base, last2, last],
            index=[
                "Mean for 2009Q4 to 2019Q4",
                f"{rate[col].index[-2]}",
                f"{rate[col].index[-1]}",
            ],
        )
    rates = pd.DataFrame(rate).dropna(how="all", axis=0)
    changed = pd.DataFrame(change).T

    # plotting
    common = {
        "lfooter": f"Australia. Job Vacancy Rate = Job Vacancies (Original series) "
        + "/ Number Employed (Seasonally adjusted) * 100. ",
        "rfooter": f"{source} {detail_lf_cat}",
        "show": SHOW,
    }

    # plot rates individually by sector
    for col in rates.columns:
        title = textwrap.fill(f"Vacancies as a % Working in {col}", width=60)
        line_plot(
            rates[col],
            title=title,
            ylabel="Job Vacancy Rate (%)",
            **common,
        )

    # plot latest versus mean 2009-2019
    ax = changed.sort_values(changed.columns[0]).plot.barh()
    ax.tick_params(axis="y", which="major", labelsize=10)
    finalise_plot(
        ax,
        title="Job Vacancy as % of Number Employed",
        ylabel=None,
        xlabel="Per cent",
        legend={"loc": "best", "fontsize": 9},
        **common,
    )

    # plot latest as a multiple of the mean 2009-2019
    multi = changed[changed.columns[-1]] / changed[changed.columns[0]]
    ax = multi.sort_values().plot.barh()
    ax.tick_params(axis="y", which="major", labelsize=10)
    finalise_plot(
        ax,
        title=f"{changed.columns[-1]} Job Vacancy Rate v 2009-2019 mean",
        ylabel=None,
        xlabel=f"Latest rate as a multiple of the 2009Q4-2019Q4 ave.",
        axvline={"x": 1, "color": "black", "linestyle": "--", "lw": 0.75},
        **common,
    )


jv_rates_by_sector()

In [6]:
def plot_industry() -> None:
    """Plot industry data."""

    stype = "Original"
    selector = {
        stype: metacol.stype,
        "^Job Vacancies": metacol.did,
        "6354004": metacol.table,
    }

    common: dict[str, Any] = {
        "selector": selector,
        "rfooter": source,
        "regex": True,
        "lfooter": f"Australia. {stype.capitalize()} series. ",
        "show": SHOW,
    }

    plot_rows_individually(
        abs_dict,
        meta,
        plot_function=plot_covid_recovery,
        tags="Covid Recovery long run",
        start_r="2000Q1",
        end_r="2019Q4",
        **common,
    )


plot_industry()

### State charts

In [7]:
def plot_state() -> None:
    """Plot state data."""

    stype = "Original"
    selector = {
        stype: metacol.stype,
        "^Job Vacancies": metacol.did,
        "Original": metacol.stype,
        "6354001": metacol.table,
    }

    common: dict[str, Any] = {
        "selector": selector,
        "regex": True,
        "rfooter": source,
        "lfooter": f"Australia. {stype.capitalize()} series. ",
        "show": SHOW,
    }

    plot_rows_individually(
        abs_dict,
        meta,
        plot_function=plot_covid_recovery,
        tags="Covid Recovery",
        start_r="2000Q1",
        end_r="2019Q4",
        **common,
    )


plot_state()

## Finished

In [8]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Thu Jan 09 2025 12:24:46

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.31.0

readabs: 0.0.21
pandas : 2.2.3

Watermark: 2.5.0



In [9]:
print("Finished")

Finished
